<a href="https://colab.research.google.com/github/harshadindigal/PythonProjects/blob/master/IMDB_Move_Ratings_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
import tensorflow_datasets as tfds
imdb,info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

In [0]:
import numpy as np
train,test = imdb['train'],imdb['test']


In [0]:
training_sequences = []
training_labels = []
testing_sequences = []
testing_labels = []

for s,l in train:
  training_sequences.append(str(s.numpy()))
  training_labels.append(l.numpy())  
for s,l in test:
    testing_sequences.append(str(s.numpy()))
    testing_labels.append(l.numpy())


In [8]:
print(training_sequences[1])

b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the storyline is awesome is very unique and i'm sure u are going to like it. Jenna amazed us once more and no wonder the movie won so many awards. Her make-up and wardrobe is very very sexy and the girls on girls scene is amazing. specially the one where she looks like an angel. It's a must see and i hope u share my interests"


In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


```
#Hyperparameters for ease of use
```

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size,oov_token = oov_tok)
tokenizer.fit_on_texts(training_sequences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sequences)
padded = pad_sequences(sequences, maxlen = max_length,truncating = trunc_type)


testing_sequence = tokenizer.texts_to_sequences(testing_sequences)
testing_padded = pad_sequences(testing_sequence, maxlen = max_length)


In [0]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model_2 = model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [28]:
model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.summary() 
num_epochs = 10
model_1.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 176us/sample - loss: 0.6066 - acc: 0.6989 - val_loss: 0.4941 - val_acc: 0.8368
Epoch 2/10
25000/25000 [==============================] 

In [29]:
model_2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.summary() 
num_epochs = 10
model_2.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 179us/sample - loss: 0.1288 - acc: 0.9632 - val_loss: 0.5866 - val_acc: 0.8219
Epoch 2/10
25000/25000 [==============================] 

In [30]:
e = model_1.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

In [0]:
import io
out_v = io.open('vecs.tsv','w',encoding = 'utf-8')
out_m = io.open('meta.tsv','w',encoding = 'utf-8')

for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')